In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from math import sqrt
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
train=pd.read_csv(r'train.csv')
test=pd.read_csv(r'test.csv')

x_train = train
x_train['Date'] = pd.to_datetime(x_train['Date'], errors='coerce')
x_train


# x_train['Season'] = x_train['Date'].dt.month
# x_train['Season']=x_train['Season'].replace([4,5,6,7,8,9],1)
# x_train['Season']=x_train['Season'].replace([1,2,3,10,11,12],0)
# x_train['Weekend'] = x_train['Date'].dt.dayofweek
# x_train['Weekend']=x_train['Weekend'].replace([0,1,2,3,4,5,6],[0,0,0,0,1,1,0])
# x_train.drop(['Date'],inplace=True,axis=1)

x_train_A = x_train[x_train['Procedure']=='A']
x_train_B = x_train[x_train['Procedure']=='B']
x_train_C = x_train[x_train['Procedure']=='C']

# P_1 = [1 if x is 'A' else 0 for x in x_train['Procedure']]
# P_2 = [1 if x is 'B' else 0 for x in x_train['Procedure']]
# P_3 = [1 if x is 'C' else 0 for x in x_train['Procedure']]
# x_train['P_1'] = P_1
# x_train['P_2'] = P_2
# x_train['P_3'] = P_3
y_train_A = x_train_A['n_Procedure']
y_train_B = x_train_B['n_Procedure']
y_train_C = x_train_B['n_Procedure']
x_train_A.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)
x_train_B.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)
x_train_C.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)

# scaler = StandardScaler()
# x_train_A=scaler.fit_transform(x_train_A)

# x_train_B=scaler.fit_transform(x_train_B)
# x_train_C=scaler.fit_transform(x_train_C)
x_train_A=np.array(x_train_A)
x_train_B=np.array(x_train_B)
x_train_C=np.array(x_train_C)
print(x_train_B.shape)
x_train_A = np.reshape(x_train_A, (x_train_A.shape[0], 1, x_train_A.shape[1]))
x_train_B = np.reshape(x_train_B, (x_train_B.shape[0], 1, x_train_B.shape[1]))
x_train_C = np.reshape(x_train_C, (x_train_C.shape[0], 1, x_train_C.shape[1]))

print(np.shape(x_train_A))

k = 1
x_A,y_A = x_train_A[:-k],y_train_A[:-k]
x_t_A,y_t_A = x_train_A[-k:],y_train_A[-k:]
print(x_A.shape)


x_B,y_B = x_train_B[:-k],y_train_B[:-k]
x_t_B,y_t_B = x_train_B[-k:],y_train_B[-k:]
print(x_B.shape)


x_C,y_C = x_train_C[:-k],y_train_C[:-k]
x_t_C,y_t_C = x_train_C[-k:],y_train_C[-k:]
print(x_C.shape)

(326, 2)
(326, 1, 2)
(325, 1, 2)
(325, 1, 2)
(325, 1, 2)


/home/akash/anaconda3/envs/my_env/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [2]:
# mu = scaler.mu

# std = np.sqrt(scaler.var_)
# mu = scaler.mean_
# std,mu

In [3]:
import math
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM,Bidirectional
def train_pred_eval_model(x_train_scaled, \
                          y_train_scaled, \
                          x_test_scaled, \
                          y_test, \
                          lstm_units, \
                          dropout_prob, \
                          optimizer, \
                          epochs, \
                          batch_size):
    '''
    Train model, do prediction, scale back to original range and do 
    evaluation
    Use LSTM here.
    Returns rmse, mape and predicted values
    Inputs
        x_train_scaled  : e.g. x_train_scaled.shape=(451, 9, 1). 
                          Here we are using the past 9 values to  
                          predict the next value
        y_train_scaled  : e.g. y_train_scaled.shape=(451, 1)
        x_test_scaled   : use this to do predictions 
        y_test          : actual value of the predictions
        mu_test_list    : list of the means. Same length as 
                          x_test_scaled and y_test
        std_test_list   : list of the std devs. Same length as 
                          x_test_scaled and y_test
        lstm_units      : dimensionality of the output space
        dropout_prob    : fraction of the units to drop for the 
                          linear transformation of the inputs
        optimizer       : optimizer for model.compile()
        epochs          : epochs for model.fit()
        batch_size      : batch size for model.fit()
    Outputs
        rmse            : root mean square error
        mape            : mean absolute percentage error
        est             : predictions
    '''
    # Create the LSTM network
    model = Sequential()
#     print(x_train_scaled.shape,y_train_scaled.shape,(x_train_scaled.shape[1],1))
    model.add((LSTM(units=lstm_units,
                   return_sequences=True, 
                   input_shape=(1,x_train_scaled.shape[2]))))
    # Add dropput with a probability of 0.5
    model.add(Dropout(dropout_prob)) 
    model.add(Bidirectional(LSTM(units=lstm_units)))
    # Add dropput with a probability of 0.5
    model.add(Dropout(dropout_prob)) 
    model.add(Dense(1))
    # Compile and fit the LSTM network
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(x_train_scaled, y_train_scaled, epochs=epochs,   
              batch_size=batch_size, verbose=1)
#     est=[]
#     # Do prediction
#     est = model.predict(x_test_scaled)
#     est = (est_scaled * np.array(std_test_list).reshape(-1,1)) +  np.array(mu_test_list).reshape(-1,1)
    
    # Calculate RMSE and MAPE
#     rmse = math.sqrt(mean_squared_error(y_test, est))
#     print(rmse)
#     print(np.exp(-rmse))
#     return rmse, est,model
    return model

Using TensorFlow backend.
/home/akash/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akash/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akash/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akash/anaconda3/e

In [4]:
 val_A=train_pred_eval_model(x_A,y_A, x_t_A,y_t_A,5,0.5,'adam',1000,10)                            

Epoch 1/1000
325/325 [==============================] - 2s 5ms/step - loss: 33.6827
Epoch 2/1000
325/325 [==============================] - 0s 364us/step - loss: 31.6926
Epoch 3/1000
325/325 [==============================] - 0s 357us/step - loss: 29.7492
Epoch 4/1000
325/325 [==============================] - 0s 346us/step - loss: 27.6486
Epoch 5/1000
325/325 [==============================] - 0s 351us/step - loss: 25.7763
Epoch 6/1000
325/325 [==============================] - 0s 351us/step - loss: 24.0338
Epoch 7/1000
325/325 [==============================] - 0s 380us/step - loss: 21.5889
Epoch 8/1000
325/325 [==============================] - 0s 359us/step - loss: 19.4481
Epoch 9/1000
325/325 [==============================] - 0s 407us/step - loss: 16.7600
Epoch 10/1000
325/325 [==============================] - 0s 332us/step - loss: 14.7474
Epoch 11/1000
325/325 [==============================] - 0s 352us/step - loss: 13.6872
Epoch 12/1000
325/325 [==============================]

325/325 [==============================] - 0s 346us/step - loss: 6.6858
Epoch 97/1000
325/325 [==============================] - 0s 321us/step - loss: 6.7527
Epoch 98/1000
325/325 [==============================] - 0s 315us/step - loss: 5.9667
Epoch 99/1000
325/325 [==============================] - 0s 314us/step - loss: 5.9943
Epoch 100/1000
325/325 [==============================] - 0s 343us/step - loss: 6.1796
Epoch 101/1000
325/325 [==============================] - 0s 349us/step - loss: 6.5735
Epoch 102/1000
325/325 [==============================] - 0s 318us/step - loss: 5.9743
Epoch 103/1000
325/325 [==============================] - 0s 322us/step - loss: 6.8145
Epoch 104/1000
325/325 [==============================] - 0s 340us/step - loss: 6.3389
Epoch 105/1000
325/325 [==============================] - 0s 350us/step - loss: 6.4872
Epoch 106/1000
325/325 [==============================] - 0s 350us/step - loss: 6.1992
Epoch 107/1000
325/325 [==============================] - 0s 

325/325 [==============================] - 0s 349us/step - loss: 5.9204
Epoch 191/1000
325/325 [==============================] - 0s 329us/step - loss: 5.7144
Epoch 192/1000
325/325 [==============================] - 0s 336us/step - loss: 5.9925
Epoch 193/1000
325/325 [==============================] - 0s 323us/step - loss: 6.0859
Epoch 194/1000
325/325 [==============================] - 0s 312us/step - loss: 6.0493
Epoch 195/1000
325/325 [==============================] - 0s 316us/step - loss: 5.6637
Epoch 196/1000
325/325 [==============================] - 0s 318us/step - loss: 5.8213
Epoch 197/1000
325/325 [==============================] - 0s 315us/step - loss: 5.6856
Epoch 198/1000
325/325 [==============================] - 0s 329us/step - loss: 5.5462
Epoch 199/1000
325/325 [==============================] - 0s 318us/step - loss: 5.9879
Epoch 200/1000
325/325 [==============================] - 0s 363us/step - loss: 5.5725
Epoch 201/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 345us/step - loss: 5.2123
Epoch 285/1000
325/325 [==============================] - 0s 330us/step - loss: 5.0456
Epoch 286/1000
325/325 [==============================] - 0s 330us/step - loss: 5.3781
Epoch 287/1000
325/325 [==============================] - 0s 313us/step - loss: 5.4909
Epoch 288/1000
325/325 [==============================] - 0s 320us/step - loss: 5.3166
Epoch 289/1000
325/325 [==============================] - 0s 353us/step - loss: 5.2586
Epoch 290/1000
325/325 [==============================] - 0s 485us/step - loss: 5.7511
Epoch 291/1000
325/325 [==============================] - 0s 421us/step - loss: 5.0463
Epoch 292/1000
325/325 [==============================] - 0s 338us/step - loss: 5.2421
Epoch 293/1000
325/325 [==============================] - 0s 402us/step - loss: 5.2797
Epoch 294/1000
325/325 [==============================] - 0s 492us/step - loss: 5.2804
Epoch 295/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 394us/step - loss: 5.2882
Epoch 379/1000
325/325 [==============================] - 0s 371us/step - loss: 5.2353
Epoch 380/1000
325/325 [==============================] - 0s 369us/step - loss: 5.1218
Epoch 381/1000
325/325 [==============================] - 0s 419us/step - loss: 4.8971
Epoch 382/1000
325/325 [==============================] - 0s 384us/step - loss: 5.3587
Epoch 383/1000
325/325 [==============================] - 0s 312us/step - loss: 4.9900
Epoch 384/1000
325/325 [==============================] - 0s 452us/step - loss: 5.0683
Epoch 385/1000
325/325 [==============================] - 0s 317us/step - loss: 5.0760
Epoch 386/1000
325/325 [==============================] - 0s 330us/step - loss: 5.2782
Epoch 387/1000
325/325 [==============================] - 0s 385us/step - loss: 5.0988
Epoch 388/1000
325/325 [==============================] - 0s 365us/step - loss: 5.0329
Epoch 389/1000
325/325 [==============================] - 

...

325/325 [==============================] - 0s 283us/step - loss: 4.9703
Epoch 567/1000
325/325 [==============================] - 0s 276us/step - loss: 5.0248
Epoch 568/1000
325/325 [==============================] - 0s 274us/step - loss: 4.9881
Epoch 569/1000
325/325 [==============================] - 0s 289us/step - loss: 4.8891
Epoch 570/1000
325/325 [==============================] - 0s 276us/step - loss: 4.9160
Epoch 571/1000
325/325 [==============================] - 0s 282us/step - loss: 4.9923
Epoch 572/1000
325/325 [==============================] - 0s 282us/step - loss: 4.8853
Epoch 573/1000
325/325 [==============================] - 0s 277us/step - loss: 4.8100
Epoch 574/1000
325/325 [==============================] - 0s 279us/step - loss: 4.7948
Epoch 575/1000
325/325 [==============================] - 0s 294us/step - loss: 4.9003
Epoch 576/1000
325/325 [==============================] - 0s 299us/step - loss: 4.8170
Epoch 577/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 276us/step - loss: 4.8310
Epoch 661/1000
325/325 [==============================] - 0s 292us/step - loss: 4.8687
Epoch 662/1000
325/325 [==============================] - 0s 278us/step - loss: 4.9055
Epoch 663/1000
325/325 [==============================] - 0s 347us/step - loss: 4.9111
Epoch 664/1000
325/325 [==============================] - 0s 265us/step - loss: 4.8176
Epoch 665/1000
325/325 [==============================] - 0s 368us/step - loss: 4.9508
Epoch 666/1000
325/325 [==============================] - 0s 304us/step - loss: 4.8592
Epoch 667/1000
325/325 [==============================] - 0s 277us/step - loss: 4.8297
Epoch 668/1000
325/325 [==============================] - 0s 279us/step - loss: 4.9202
Epoch 669/1000
325/325 [==============================] - 0s 280us/step - loss: 4.9264
Epoch 670/1000
325/325 [==============================] - 0s 275us/step - loss: 4.8930
Epoch 671/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 283us/step - loss: 4.8857
Epoch 755/1000
325/325 [==============================] - 0s 280us/step - loss: 4.8626
Epoch 756/1000
325/325 [==============================] - 0s 276us/step - loss: 4.9243
Epoch 757/1000
325/325 [==============================] - 0s 304us/step - loss: 4.9008
Epoch 758/1000
325/325 [==============================] - 0s 293us/step - loss: 4.8605
Epoch 759/1000
325/325 [==============================] - 0s 525us/step - loss: 4.8488
Epoch 760/1000
325/325 [==============================] - 0s 441us/step - loss: 4.8370
Epoch 761/1000
325/325 [==============================] - 0s 278us/step - loss: 4.8724
Epoch 762/1000
325/325 [==============================] - 0s 288us/step - loss: 4.8975
Epoch 763/1000
325/325 [==============================] - 0s 276us/step - loss: 4.7882
Epoch 764/1000
325/325 [==============================] - 0s 281us/step - loss: 4.9074
Epoch 765/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 325us/step - loss: 4.9091
Epoch 849/1000
325/325 [==============================] - 0s 430us/step - loss: 4.8036
Epoch 850/1000
325/325 [==============================] - 0s 324us/step - loss: 4.8818
Epoch 851/1000
325/325 [==============================] - 0s 443us/step - loss: 4.8434
Epoch 852/1000
325/325 [==============================] - 0s 295us/step - loss: 4.8562
Epoch 853/1000
325/325 [==============================] - 0s 278us/step - loss: 4.8559
Epoch 854/1000
325/325 [==============================] - 0s 300us/step - loss: 4.8845
Epoch 855/1000
325/325 [==============================] - 0s 395us/step - loss: 4.8549
Epoch 856/1000
325/325 [==============================] - 0s 282us/step - loss: 4.8185
Epoch 857/1000
325/325 [==============================] - 0s 279us/step - loss: 4.8565
Epoch 858/1000
325/325 [==============================] - 0s 354us/step - loss: 4.8972
Epoch 859/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 308us/step - loss: 4.8192
Epoch 943/1000
325/325 [==============================] - 0s 312us/step - loss: 4.8465
Epoch 944/1000
325/325 [==============================] - 0s 334us/step - loss: 4.8419
Epoch 945/1000
325/325 [==============================] - 0s 355us/step - loss: 4.8992
Epoch 946/1000
325/325 [==============================] - 0s 292us/step - loss: 4.8224
Epoch 947/1000
325/325 [==============================] - 0s 497us/step - loss: 4.8471
Epoch 948/1000
325/325 [==============================] - 0s 513us/step - loss: 4.8841
Epoch 949/1000
325/325 [==============================] - 0s 402us/step - loss: 4.8093
Epoch 950/1000
325/325 [==============================] - 0s 357us/step - loss: 4.8836
Epoch 951/1000
325/325 [==============================] - 0s 311us/step - loss: 4.8374
Epoch 952/1000
325/325 [==============================] - 0s 333us/step - loss: 4.8186
Epoch 953/1000
325/325 [==============================] - 

In [5]:
# pred_A = val_A[1]
# pred1_A = np.round(pred_A)

In [6]:
# pred1_A

In [7]:
# z_A=y_t_A
# r = math.sqrt(mean_squared_error(z_A,pred1_A))
# r

In [8]:
# print(np.exp(-r))

In [9]:
val_B=train_pred_eval_model(x_B,y_B,x_t_B,y_t_B,3,0.7,'adam',1000,10)


Epoch 1/1000
325/325 [==============================] - 2s 6ms/step - loss: 9.8857
Epoch 2/1000
325/325 [==============================] - 0s 288us/step - loss: 9.6182
Epoch 3/1000
325/325 [==============================] - 0s 281us/step - loss: 9.3271
Epoch 4/1000
325/325 [==============================] - 0s 284us/step - loss: 8.9678
Epoch 5/1000
325/325 [==============================] - 0s 279us/step - loss: 8.6618
Epoch 6/1000
325/325 [==============================] - 0s 292us/step - loss: 8.3088
Epoch 7/1000
325/325 [==============================] - 0s 296us/step - loss: 7.8963
Epoch 8/1000
325/325 [==============================] - 0s 300us/step - loss: 7.3861
Epoch 9/1000
325/325 [==============================] - 0s 305us/step - loss: 7.0378
Epoch 10/1000
325/325 [==============================] - 0s 287us/step - loss: 6.7215
Epoch 11/1000
325/325 [==============================] - 0s 292us/step - loss: 6.4007
Epoch 12/1000
325/325 [==============================] - 0s 279us

325/325 [==============================] - 0s 288us/step - loss: 3.0992
Epoch 97/1000
325/325 [==============================] - 0s 314us/step - loss: 3.2528
Epoch 98/1000
325/325 [==============================] - 0s 297us/step - loss: 3.2521
Epoch 99/1000
325/325 [==============================] - 0s 288us/step - loss: 3.3343
Epoch 100/1000
325/325 [==============================] - 0s 325us/step - loss: 3.3255
Epoch 101/1000
325/325 [==============================] - 0s 298us/step - loss: 3.3324
Epoch 102/1000
325/325 [==============================] - 0s 319us/step - loss: 3.2631
Epoch 103/1000
325/325 [==============================] - 0s 293us/step - loss: 3.3770
Epoch 104/1000
325/325 [==============================] - 0s 312us/step - loss: 3.1614
Epoch 105/1000
325/325 [==============================] - 0s 303us/step - loss: 3.3901
Epoch 106/1000
325/325 [==============================] - 0s 292us/step - loss: 3.3824
Epoch 107/1000
325/325 [==============================] - 0s 

325/325 [==============================] - 0s 280us/step - loss: 3.0342
Epoch 191/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9398
Epoch 192/1000
325/325 [==============================] - 0s 280us/step - loss: 3.0810
Epoch 193/1000
325/325 [==============================] - 0s 279us/step - loss: 3.0319
Epoch 194/1000
325/325 [==============================] - 0s 281us/step - loss: 3.0564
Epoch 195/1000
325/325 [==============================] - 0s 276us/step - loss: 2.9799
Epoch 196/1000
325/325 [==============================] - 0s 281us/step - loss: 3.0135
Epoch 197/1000
325/325 [==============================] - 0s 281us/step - loss: 3.0017
Epoch 198/1000
325/325 [==============================] - 0s 287us/step - loss: 3.0074
Epoch 199/1000
325/325 [==============================] - 0s 294us/step - loss: 2.9963
Epoch 200/1000
325/325 [==============================] - 0s 295us/step - loss: 2.9820
Epoch 201/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 298us/step - loss: 2.9659
Epoch 285/1000
325/325 [==============================] - 0s 321us/step - loss: 2.9702
Epoch 286/1000
325/325 [==============================] - 0s 384us/step - loss: 2.9962
Epoch 287/1000
325/325 [==============================] - 0s 389us/step - loss: 2.9881
Epoch 288/1000
325/325 [==============================] - 0s 391us/step - loss: 2.9638
Epoch 289/1000
325/325 [==============================] - 0s 311us/step - loss: 2.9463
Epoch 290/1000
325/325 [==============================] - 0s 411us/step - loss: 2.9829
Epoch 291/1000
325/325 [==============================] - 0s 281us/step - loss: 2.9768
Epoch 292/1000
325/325 [==============================] - 0s 290us/step - loss: 2.9753
Epoch 293/1000
325/325 [==============================] - 0s 289us/step - loss: 2.9582
Epoch 294/1000
325/325 [==============================] - 0s 361us/step - loss: 2.9660
Epoch 295/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 281us/step - loss: 2.9682
Epoch 379/1000
325/325 [==============================] - 0s 290us/step - loss: 2.9781
Epoch 380/1000
325/325 [==============================] - 0s 286us/step - loss: 2.9757
Epoch 381/1000
325/325 [==============================] - 0s 278us/step - loss: 2.9628
Epoch 382/1000
325/325 [==============================] - 0s 279us/step - loss: 2.9644
Epoch 383/1000
325/325 [==============================] - 0s 296us/step - loss: 2.9681
Epoch 384/1000
325/325 [==============================] - 0s 282us/step - loss: 2.9691
Epoch 385/1000
325/325 [==============================] - 0s 307us/step - loss: 2.9767
Epoch 386/1000
325/325 [==============================] - 0s 280us/step - loss: 2.9765
Epoch 387/1000
325/325 [==============================] - 0s 283us/step - loss: 2.9666
Epoch 388/1000
325/325 [==============================] - 0s 278us/step - loss: 2.9686
Epoch 389/1000
325/325 [==============================] - 

...

325/325 [==============================] - 0s 370us/step - loss: 2.9654
Epoch 567/1000
325/325 [==============================] - 0s 330us/step - loss: 2.9673
Epoch 568/1000
325/325 [==============================] - 0s 337us/step - loss: 2.9678
Epoch 569/1000
325/325 [==============================] - 0s 330us/step - loss: 2.9654
Epoch 570/1000
325/325 [==============================] - 0s 331us/step - loss: 2.9761
Epoch 571/1000
325/325 [==============================] - 0s 370us/step - loss: 2.9672
Epoch 572/1000
325/325 [==============================] - 0s 385us/step - loss: 2.9662
Epoch 573/1000
325/325 [==============================] - 0s 372us/step - loss: 2.9647
Epoch 574/1000
325/325 [==============================] - 0s 377us/step - loss: 2.9714
Epoch 575/1000
325/325 [==============================] - 0s 337us/step - loss: 2.9650
Epoch 576/1000
325/325 [==============================] - 0s 348us/step - loss: 2.9646
Epoch 577/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 302us/step - loss: 2.9639
Epoch 661/1000
325/325 [==============================] - 0s 304us/step - loss: 2.9655
Epoch 662/1000
325/325 [==============================] - 0s 414us/step - loss: 2.9650
Epoch 663/1000
325/325 [==============================] - 0s 379us/step - loss: 2.9629
Epoch 664/1000
325/325 [==============================] - 0s 384us/step - loss: 2.9646
Epoch 665/1000
325/325 [==============================] - 0s 364us/step - loss: 2.9608
Epoch 666/1000
325/325 [==============================] - 0s 356us/step - loss: 2.9655
Epoch 667/1000
325/325 [==============================] - 0s 364us/step - loss: 2.9650
Epoch 668/1000
325/325 [==============================] - 0s 357us/step - loss: 2.9661
Epoch 669/1000
325/325 [==============================] - 0s 309us/step - loss: 2.9672
Epoch 670/1000
325/325 [==============================] - 0s 286us/step - loss: 2.9654
Epoch 671/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 315us/step - loss: 2.9701
Epoch 755/1000
325/325 [==============================] - 0s 789us/step - loss: 2.9673
Epoch 756/1000
325/325 [==============================] - 0s 788us/step - loss: 2.9653
Epoch 757/1000
325/325 [==============================] - 0s 515us/step - loss: 2.9650
Epoch 758/1000
325/325 [==============================] - 0s 316us/step - loss: 2.9654
Epoch 759/1000
325/325 [==============================] - 0s 373us/step - loss: 2.9648
Epoch 760/1000
325/325 [==============================] - 0s 387us/step - loss: 2.9669
Epoch 761/1000
325/325 [==============================] - 0s 410us/step - loss: 2.9625
Epoch 762/1000
325/325 [==============================] - 0s 448us/step - loss: 2.9611
Epoch 763/1000
325/325 [==============================] - 0s 466us/step - loss: 2.9655
Epoch 764/1000
325/325 [==============================] - 0s 617us/step - loss: 2.9666
Epoch 765/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 295us/step - loss: 2.9673
Epoch 849/1000
325/325 [==============================] - 0s 296us/step - loss: 2.9622
Epoch 850/1000
325/325 [==============================] - 0s 315us/step - loss: 2.9611
Epoch 851/1000
325/325 [==============================] - 0s 290us/step - loss: 2.9685
Epoch 852/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9730
Epoch 853/1000
325/325 [==============================] - 0s 365us/step - loss: 2.9675
Epoch 854/1000
325/325 [==============================] - 0s 280us/step - loss: 2.9634
Epoch 855/1000
325/325 [==============================] - 0s 403us/step - loss: 2.9696
Epoch 856/1000
325/325 [==============================] - 0s 330us/step - loss: 2.9598
Epoch 857/1000
325/325 [==============================] - 0s 281us/step - loss: 2.9619
Epoch 858/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9693
Epoch 859/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 349us/step - loss: 2.9702
Epoch 943/1000
325/325 [==============================] - 0s 358us/step - loss: 2.9648
Epoch 944/1000
325/325 [==============================] - 0s 322us/step - loss: 2.9594
Epoch 945/1000
325/325 [==============================] - 0s 331us/step - loss: 2.9707
Epoch 946/1000
325/325 [==============================] - 0s 295us/step - loss: 2.9631
Epoch 947/1000
325/325 [==============================] - 0s 294us/step - loss: 2.9630
Epoch 948/1000
325/325 [==============================] - 0s 328us/step - loss: 2.9632
Epoch 949/1000
325/325 [==============================] - 0s 334us/step - loss: 2.9644
Epoch 950/1000
325/325 [==============================] - 0s 326us/step - loss: 2.9641
Epoch 951/1000
325/325 [==============================] - 0s 318us/step - loss: 2.9608
Epoch 952/1000
325/325 [==============================] - 0s 359us/step - loss: 2.9608
Epoch 953/1000
325/325 [==============================] - 

In [10]:
# pred_B = val_B[1]
# pred1_B = np.round(pred_B)
# pred1_B


In [11]:
# z_B=y_t_B
# r = math.sqrt(mean_squared_error(z_B,pred1_B))
# r
# print(np.exp(-r))

In [12]:
x_C

array([[[27,  1]],

       [[21,  0]],

       [[20,  1]],

       [[27,  0]],

       [[22,  0]],

       [[23,  0]],

       [[24,  2]],

       [[20,  2]],

       [[25,  2]],

       [[25,  2]],

       [[25,  2]],

       [[21,  2]],

       [[26,  2]],

       [[23,  0]],

       [[28,  0]],

       [[19,  0]],

       [[20,  0]],

       [[24,  0]],

       [[24,  0]],

       [[27,  0]],

       [[30,  0]],

       [[19,  0]],

       [[19,  0]],

       [[19,  0]],

       [[20,  0]],

       [[23,  0]],

       [[23,  0]],

       [[24,  0]],

       [[25,  0]],

       [[28,  0]],

       [[21,  0]],

       [[23,  0]],

       [[19,  1]],

       [[22,  2]],

       [[21,  2]],

       [[19,  0]],

       [[21,  0]],

       [[21,  0]],

       [[21,  0]],

       [[28,  2]],

       [[21,  2]],

       [[20,  2]],

       [[25,  0]],

       [[20,  0]],

       [[29,  0]],

       [[29,  0]],

       [[21,  0]],

       [[26,  2]],

       [[20,  2]],

       [[20,  2]],



In [13]:
val_C=train_pred_eval_model(x_C,y_C,x_t_C,y_t_C,5,0.5,'adam',1000,10)


Epoch 1/1000
325/325 [==============================] - 2s 8ms/step - loss: 10.3466
Epoch 2/1000
325/325 [==============================] - 0s 336us/step - loss: 9.6550
Epoch 3/1000
325/325 [==============================] - 0s 421us/step - loss: 8.9556
Epoch 4/1000
325/325 [==============================] - 0s 348us/step - loss: 8.4522
Epoch 5/1000
325/325 [==============================] - 0s 364us/step - loss: 7.8729
Epoch 6/1000
325/325 [==============================] - 0s 331us/step - loss: 7.3726
Epoch 7/1000
325/325 [==============================] - 0s 340us/step - loss: 6.9103
Epoch 8/1000
325/325 [==============================] - 0s 351us/step - loss: 6.4780
Epoch 9/1000
325/325 [==============================] - 0s 355us/step - loss: 5.9427
Epoch 10/1000
325/325 [==============================] - 0s 337us/step - loss: 5.4753
Epoch 11/1000
325/325 [==============================] - 0s 364us/step - loss: 5.0439
Epoch 12/1000
325/325 [==============================] - 0s 348u

325/325 [==============================] - 0s 352us/step - loss: 3.1006
Epoch 97/1000
325/325 [==============================] - 0s 325us/step - loss: 3.2365
Epoch 98/1000
325/325 [==============================] - 0s 355us/step - loss: 2.9099
Epoch 99/1000
325/325 [==============================] - 0s 362us/step - loss: 3.1595
Epoch 100/1000
325/325 [==============================] - 0s 343us/step - loss: 3.1578
Epoch 101/1000
325/325 [==============================] - 0s 341us/step - loss: 3.1221
Epoch 102/1000
325/325 [==============================] - 0s 354us/step - loss: 3.2743
Epoch 103/1000
325/325 [==============================] - 0s 341us/step - loss: 3.2338
Epoch 104/1000
325/325 [==============================] - 0s 354us/step - loss: 3.1870
Epoch 105/1000
325/325 [==============================] - 0s 367us/step - loss: 3.3234
Epoch 106/1000
325/325 [==============================] - 0s 335us/step - loss: 3.2428
Epoch 107/1000
325/325 [==============================] - 0s 

325/325 [==============================] - 0s 587us/step - loss: 3.2188
Epoch 191/1000
325/325 [==============================] - 0s 433us/step - loss: 3.1363
Epoch 192/1000
325/325 [==============================] - 0s 387us/step - loss: 3.1596
Epoch 193/1000
325/325 [==============================] - 0s 551us/step - loss: 3.0715
Epoch 194/1000
325/325 [==============================] - 0s 427us/step - loss: 2.9420
Epoch 195/1000
325/325 [==============================] - 0s 352us/step - loss: 3.0674
Epoch 196/1000
325/325 [==============================] - 0s 369us/step - loss: 3.1538
Epoch 197/1000
325/325 [==============================] - 0s 391us/step - loss: 3.0151
Epoch 198/1000
325/325 [==============================] - 0s 351us/step - loss: 3.0587
Epoch 199/1000
325/325 [==============================] - 0s 533us/step - loss: 3.1147
Epoch 200/1000
325/325 [==============================] - 0s 394us/step - loss: 3.0639
Epoch 201/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 421us/step - loss: 3.0005
Epoch 285/1000
325/325 [==============================] - 0s 360us/step - loss: 3.0709
Epoch 286/1000
325/325 [==============================] - 0s 426us/step - loss: 3.0264
Epoch 287/1000
325/325 [==============================] - 0s 338us/step - loss: 3.0430
Epoch 288/1000
325/325 [==============================] - 0s 333us/step - loss: 3.0521
Epoch 289/1000
325/325 [==============================] - 0s 438us/step - loss: 3.0556
Epoch 290/1000
325/325 [==============================] - 0s 357us/step - loss: 3.0169
Epoch 291/1000
325/325 [==============================] - 0s 420us/step - loss: 3.0417
Epoch 292/1000
325/325 [==============================] - 0s 322us/step - loss: 3.0050
Epoch 293/1000
325/325 [==============================] - 0s 320us/step - loss: 3.0223
Epoch 294/1000
325/325 [==============================] - 0s 364us/step - loss: 3.0125
Epoch 295/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 372us/step - loss: 2.9272
Epoch 379/1000
325/325 [==============================] - 0s 533us/step - loss: 3.0370
Epoch 380/1000
325/325 [==============================] - 0s 476us/step - loss: 3.0362
Epoch 381/1000
325/325 [==============================] - 0s 346us/step - loss: 2.9853
Epoch 382/1000
325/325 [==============================] - 0s 360us/step - loss: 2.9980
Epoch 383/1000
325/325 [==============================] - 0s 403us/step - loss: 3.0329
Epoch 384/1000
325/325 [==============================] - 0s 350us/step - loss: 2.9766
Epoch 385/1000
325/325 [==============================] - 0s 340us/step - loss: 2.9806
Epoch 386/1000
325/325 [==============================] - 0s 576us/step - loss: 3.0066
Epoch 387/1000
325/325 [==============================] - 0s 408us/step - loss: 2.9429
Epoch 388/1000
325/325 [==============================] - 0s 360us/step - loss: 2.9880
Epoch 389/1000
325/325 [==============================] - 

...

325/325 [==============================] - 0s 541us/step - loss: 2.9532
Epoch 567/1000
325/325 [==============================] - 0s 574us/step - loss: 2.9856
Epoch 568/1000
325/325 [==============================] - 0s 549us/step - loss: 2.9906
Epoch 569/1000
325/325 [==============================] - 0s 639us/step - loss: 2.9863
Epoch 570/1000
325/325 [==============================] - 0s 765us/step - loss: 2.9754
Epoch 571/1000
325/325 [==============================] - 0s 771us/step - loss: 3.0008
Epoch 572/1000
325/325 [==============================] - 0s 627us/step - loss: 2.9765
Epoch 573/1000
325/325 [==============================] - 0s 363us/step - loss: 2.9412
Epoch 574/1000
325/325 [==============================] - 0s 368us/step - loss: 2.9550
Epoch 575/1000
325/325 [==============================] - 0s 353us/step - loss: 2.9826
Epoch 576/1000
325/325 [==============================] - 0s 348us/step - loss: 2.9614
Epoch 577/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 318us/step - loss: 2.9837
Epoch 661/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9483
Epoch 662/1000
325/325 [==============================] - 0s 299us/step - loss: 2.9602
Epoch 663/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9769
Epoch 664/1000
325/325 [==============================] - 0s 286us/step - loss: 2.9532
Epoch 665/1000
325/325 [==============================] - 0s 287us/step - loss: 2.9945
Epoch 666/1000
325/325 [==============================] - 0s 308us/step - loss: 2.9795
Epoch 667/1000
325/325 [==============================] - 0s 285us/step - loss: 2.9577
Epoch 668/1000
325/325 [==============================] - 0s 328us/step - loss: 2.9769
Epoch 669/1000
325/325 [==============================] - 0s 318us/step - loss: 2.9562
Epoch 670/1000
325/325 [==============================] - 0s 364us/step - loss: 2.9642
Epoch 671/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 286us/step - loss: 2.9683
Epoch 755/1000
325/325 [==============================] - 0s 304us/step - loss: 2.9542
Epoch 756/1000
325/325 [==============================] - 0s 285us/step - loss: 2.9522
Epoch 757/1000
325/325 [==============================] - 0s 280us/step - loss: 2.9831
Epoch 758/1000
325/325 [==============================] - 0s 283us/step - loss: 2.9641
Epoch 759/1000
325/325 [==============================] - 0s 288us/step - loss: 2.9573
Epoch 760/1000
325/325 [==============================] - 0s 287us/step - loss: 2.9724
Epoch 761/1000
325/325 [==============================] - 0s 290us/step - loss: 2.9516
Epoch 762/1000
325/325 [==============================] - 0s 282us/step - loss: 2.9626
Epoch 763/1000
325/325 [==============================] - 0s 307us/step - loss: 2.9837
Epoch 764/1000
325/325 [==============================] - 0s 292us/step - loss: 2.9772
Epoch 765/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 326us/step - loss: 2.9736
Epoch 849/1000
325/325 [==============================] - 0s 303us/step - loss: 2.9641
Epoch 850/1000
325/325 [==============================] - 0s 285us/step - loss: 2.9897
Epoch 851/1000
325/325 [==============================] - 0s 287us/step - loss: 2.9611
Epoch 852/1000
325/325 [==============================] - 0s 301us/step - loss: 2.9602
Epoch 853/1000
325/325 [==============================] - 0s 298us/step - loss: 2.9469
Epoch 854/1000
325/325 [==============================] - 0s 311us/step - loss: 2.9731
Epoch 855/1000
325/325 [==============================] - 0s 283us/step - loss: 2.9535
Epoch 856/1000
325/325 [==============================] - 0s 317us/step - loss: 2.9565
Epoch 857/1000
325/325 [==============================] - 0s 283us/step - loss: 2.9656
Epoch 858/1000
325/325 [==============================] - 0s 288us/step - loss: 2.9556
Epoch 859/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 287us/step - loss: 2.9528
Epoch 943/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9501
Epoch 944/1000
325/325 [==============================] - 0s 285us/step - loss: 2.9573
Epoch 945/1000
325/325 [==============================] - 0s 288us/step - loss: 2.9741
Epoch 946/1000
325/325 [==============================] - 0s 292us/step - loss: 2.9713
Epoch 947/1000
325/325 [==============================] - 0s 297us/step - loss: 2.9617
Epoch 948/1000
325/325 [==============================] - 0s 294us/step - loss: 2.9503
Epoch 949/1000
325/325 [==============================] - 0s 300us/step - loss: 2.9725
Epoch 950/1000
325/325 [==============================] - 0s 307us/step - loss: 2.9561
Epoch 951/1000
325/325 [==============================] - 0s 302us/step - loss: 2.9542
Epoch 952/1000
325/325 [==============================] - 0s 293us/step - loss: 2.9465
Epoch 953/1000
325/325 [==============================] - 

In [14]:
# pred_C = val_C[1]
# pred1_C = np.round(pred_C)
# pred1_C


In [15]:
# z_C=y_t_C
# r = math.sqrt(mean_squared_error(z_C,pred1_C))
# print(r)
# print(np.exp(-r))

In [16]:
model_A=val_A
model_B=val_B
model_C=val_C

In [17]:
test.drop(['Date'],inplace=True,axis=1)


In [18]:
print(test)

    Temperature  EventTarget
0            28            0
1            29            0
2            29            0
3            32            0
4            31            0
5            34            0
6            34            0
7            27            0
8            29            0
9            31            2
10           33            2
11           27            2
12           26            0
13           33            0
14           26            1
15           30            1
16           25            1
17           30            0
18           29            0
19           30            0
20           26            0
21           31            0
22           31            0
23           25            0
24           31            0
25           31            0
26           29            0
27           25            0
28           25            0
29           29            0
30           30            0
31           31            0
32           32            0
33           3

In [19]:
test=np.array(test)
test

array([[28,  0],
       [29,  0],
       [29,  0],
       [32,  0],
       [31,  0],
       [34,  0],
       [34,  0],
       [27,  0],
       [29,  0],
       [31,  2],
       [33,  2],
       [27,  2],
       [26,  0],
       [33,  0],
       [26,  1],
       [30,  1],
       [25,  1],
       [30,  0],
       [29,  0],
       [30,  0],
       [26,  0],
       [31,  0],
       [31,  0],
       [25,  0],
       [31,  0],
       [31,  0],
       [29,  0],
       [25,  0],
       [25,  0],
       [29,  0],
       [30,  0],
       [31,  0],
       [32,  0],
       [30,  0],
       [32,  0],
       [30,  2],
       [25,  2],
       [28,  2],
       [27,  2],
       [25,  2],
       [21,  2]])

In [20]:
test

array([[28,  0],
       [29,  0],
       [29,  0],
       [32,  0],
       [31,  0],
       [34,  0],
       [34,  0],
       [27,  0],
       [29,  0],
       [31,  2],
       [33,  2],
       [27,  2],
       [26,  0],
       [33,  0],
       [26,  1],
       [30,  1],
       [25,  1],
       [30,  0],
       [29,  0],
       [30,  0],
       [26,  0],
       [31,  0],
       [31,  0],
       [25,  0],
       [31,  0],
       [31,  0],
       [29,  0],
       [25,  0],
       [25,  0],
       [29,  0],
       [30,  0],
       [31,  0],
       [32,  0],
       [30,  0],
       [32,  0],
       [30,  2],
       [25,  2],
       [28,  2],
       [27,  2],
       [25,  2],
       [21,  2]])

In [21]:
test = np.reshape(test, (test.shape[0], 1, test.shape[1]))


In [24]:
# A=model_A.predict(test)
A=(model_A.predict(test))
B=(model_B.predict(test))
C=(model_C.predict(test))
# print(A)

In [25]:
final=[]
for i in range(len(A)):
    final.append(A[i])
    final.append(B[i])
    final.append(C[i])
df=pd.DataFrame(final,columns = ['n_Procedure'])
    

In [26]:
df

,n_Procedure
0,5.445212
1,2.650780
2,2.636186
3,5.440979
4,2.650780
...,...
118,2.650780
119,2.634454
120,5.353514
121,2.650780


In [27]:
df.to_csv('Final.csv')